In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as const
import emcee, corner, starry

In [ ]:
# Import the helper package I wrote for you
import helper

# Setup instructions

This will take ~25 MB and should be done before the lab.

```conda install starry```

OR

```pip install starry==0.3.0```

# Introduction to Exoplanet Science

Some day we are going to find a planet in another solar system that might harbour life, and we're going to want to know everything about that planet: what temperature the planet is, whether there's water in its atmosphere, etc. In the further future, we may even want to know whether there are continents and oceans on an exoplanet, and people are already writing code to model the future observations that will allow us to determine this (e.g. [Farr et al. 2018](https://iopscience.iop.org/article/10.3847/1538-3881/aad775/pdf)).

However, our images look like complete garbage because these systems are so far away. We just see a single, unresolved blob of light that contains all the starlight and the light of the planet. Here is a real Spitzer image for example:

<img src='images/WASP12b_Spitzer_Image.png' width=40%>
<center>Image Credit: Bell et al. 2019</center>

Since we can't directly see exoplanets themselves (or at least not in the vast majority of cases), we instead have to resort to indirect techniques for now (with future advances in techniques like direct imaging potentially allowing us to do this more easily or differently).

# Exoplanet Transits

When a planet passes in front of its host star, we see a dip in brightness as some starlight is blocked. Using this phoenomenon, we can infer the star-planet radius ratio.

<img src='images/640px-Kepler_6b.png' width=40%>
<center>Image Credit: Wikipedia</center>

As you can see, however, transits are normally not flat bottomed. This is due to a phoenomenon known as limb darkening https://en.wikipedia.org/wiki/Limb_darkening. This is caused by stars having a non-uniform surface brightness which makes stars appear dimmer and redder near their edges (called the limb of the star). An image of the Sun is below, showing the effect of limb darkening as well as the transit of Venus in the upper right corner.

<img src='images/720px-2012_Transit_of_Venus_from_SF.jpg' width=20%>
<center>Image Credit: Wikipedia</center>

This effect can be reasonably modelled as a quadratic function of separation from the centre of the star's visible disk of the form:

\begin{equation}
    I(\mu) = I_0[1 - c_1(1 - \mu) - c_2(1-\mu)^2]
\end{equation}

where $\mu = \sqrt{1-x^2}$ and $x$ is the normalized impact parameter ($0 \le x \le 1$).

# Introduction to starry

Thankfully there have been numerous people that have come before us, and we don't need to re-invent the wheel.

Only recently, [Luger et al. 2019](https://arxiv.org/pdf/1810.06559.pdf) wrote a package in python called $\texttt{starry}$ which will do almost everything we need analytically (and therefore very quickly). Much of the introduction to the $\texttt{starry}$ package has been drawn directly from their own tutorial which can be found at https://rodluger.github.io/starry/v0.3.0/tutorials/basics3.html

To make a star object using $\texttt{starry}$ and initialize it's limb-darkening, we do the following:

In [ ]:
star = starry.kepler.Primary()
star[1] = 0.40 # linear term
star[2] = 0.26 # quadratic term

star.show()

We’ve given the star a quadratically limb-darkened ($\texttt{udeg} = 2$). The aplitude $\texttt{amp}$ controls the overall scaling of the intensity of the object and is therefore equal to its luminosity (in arbitrary units).

To then initialize the planet, we can do the following:

In [ ]:
planet = starry.kepler.Secondary(lmax=1)  # (lmax described later)

planet.L = 0   # (described later)
planet.r = 0.1  # radius in stellar radii
planet.a = 5   # orbital semi-major axis in units of the stellar radii.
planet.inc = 90 # orbital inclination (90 degrees is an edge-on orbit)
planet.porb = 1.0  # orbital period in days
planet.prot = 1.0  # rotation period in days (synchronous if prot=porb)
planet.ecc = 0.  # eccentricity
planet.w = 30  # longitude of pericenter in degrees (meaningless for ecc=0)
planet.tref = 0  # time of transit in days

To show the lightcurve of the system, we can then do the following

In [ ]:
system = starry.kepler.System(star, planet)
time = np.linspace(-0.1, 0.1, 1000)

system.compute(time) # compute the lightcurve which will be stored within system
flux_system = system.lightcurve

plt.plot((time-planet.tref)*24*60, flux_system)
plt.xlabel("Time from transit (minutes)")
plt.ylabel("System Flux")
plt.show()

### Question 1)

Make a new planet that has the same characteristics as my favourite planet WASP-12b whose information can be found at http://exoplanet.eu/catalog/wasp-12_b/. For now, you can still use ```lmax=1``` and ```planet.L=0```. Make sure to set ```planet.prot``` to the same value as ```planet.porb```

Make a plot showing the full transit of WASP-12b with ~30 minutes of flat baseline on either side of the transit (this is what a typical observation of an exoplanet would look like).

In [ ]:
# Your code goes here



# Exoplanet Eclipses

A very similar phoenomenon occurs on the other side of the orbit when the planet passes behind the star—this is called an secondary eclipse, or just eclipse. This time though, the planet will be fully obscured by the much larger star, and we expect to see a flat bottomed lightcurve. The depth of the eclipse tells us about how bright the planet is compared to the host star, as well as the radius ratio of the two objects (which we can measure using the transit).

To set the luminosity ratio $L_p/L_*$ of the system, you use the syntax ```planet.L = myValue```

### Question 2)

Let's assume we're looking at the eclipse of WASP-12b using the Channel 2 detector of the IRAC instrument on the Spitzer Space Telescope, which has a central wavelength of 4.5 $\mu$m and a bandwidth of 1 $\mu$m.

As a moderately reasonable approximation, we'll assume that the planet and star are blackbodies, allowing us to compute the fluxes of the bodies using [Planck's Law](https://en.wikipedia.org/wiki/Planck%27s_law). Numerically integrate $B_{\lambda}$ over the channel 2 bandpass for both the planet and the star (nothing fancy here, just the np.sum($B_{\lambda}$\*d$\lambda$) that you're used to doing). Assume a dayside effective temperature of 3000 K [(Bell et al., 2019)](https://arxiv.org/pdf/1906.04742.pdf) and a stellar effective temperature of 6300 K [(Hebb et al., 2009)](https://arxiv.org/pdf/0812.3240.pdf).

Next, compute the power output by each object by multiplying by $(\pi ~\text{steradians})*\text{Radius}^2$. Be careful with your units here! Assume that the planet appears to have a radius of 1.7182 R$_{\rm Jup}$ at 4.5 $\mu$m [(Bell et al., 2019)](https://arxiv.org/pdf/1906.04742.pdf) and keep your literature value for the stellar radius.

Then compute the planet-to-star power ratio of the two objects, and assign it to ```planet.L``` (we'll assume there is no flux coming from the side facing away from the star for now). You should get a value of 0.3917%. Also update your ```planet.r``` for the the radius of the planet as seen at 4.5 $\mu$m, keeping in mind that this is in units of stellar radii.

Finally, plot the eclipse lightcurve (which should take place halfway around the orbit since we set the orbital eccentricity to zero). How does the depth of the eclipse compare to the depth of the transit?

In [ ]:
# Your code goes here




# Exoplanet Phasecurves

There are many different ways of modelling the temperature variations on the surface of an exoplanet. For almost all of the planets that current instruments are able to observe, the planets day is equal to its year—this is called synchronous rotation, and is a special case of tidal locking. In this situation, the planet has a hemisphere that is permanently illuminated by its host star (which we call the dayside), and a hemisphere that is in perpetual darkness (which we call the nightside). This strong contrast in irradiation results in a strong temperature contrast between the day and night hemispheres. This strong temperature contrast then leads to rapid winds which transport heat from the dayside toward the nightside. Complicated fluid dynamics simulations suggest that these winds should (for the most part) flow to the east (which is defined as having a spin vector aligned with the orbital spin vector).

A reasonable way to model the temperature of these highly irradiated, synchronously rotating exoplanets is just a simple Fourier series as a function of longitude. Since we are dealing with a spherical surface, we will use spherical harmonics to model the temperature map of the planet.

Above, we set ```lmax=1``` while initializing our ```planet``` object. This corresponds to a 1st order spherical harmonic map. Each term is depicted below:
<img src="images/ylms.png" width=50%>
<center>Image Credit: https://rodluger.github.io/starry/v0.3.0/tutorials/basics1.html</center>

The zeroth order term (l=0,m=0) sets the overall output of the planet, (l=1,m=0) sets the day-to-night contrast, and (l=1,m=1) sets the longitudinal offset of the hottest point on the planet (where positive values are eastward offsets). The term (l=1,m=-1) is generally assumed to be zero and is exceedingly difficult to measure anyways.

We previously set the $\ell=0$ term using ```planet.L```. To set the higher order terms, you use the syntax ```planet[l,m] = myValue```

Let's imagine our observations start at eclipse where we would be seeing the scorching hot dayside of the planet. As the planet orbits around the star, that dayside of the planet stays facing the star and we begin to see parts of the cooler nightside. Then when we get to the time of transit, we will be seeing only the cold nightside of the planet. These variations in the average temperature of the hemisphere facing us results in sinusoidal modulations in the flux coming from the planet with a maximum near eclipse and a minimum near transit. We call these variations phase variations, and we call the observation of the full orbit of the system a phasecurve.

An example image showing the full orbit phasecurve is shown below

<img src="images/phasecurve.jpg" width=70%>
<center>Image Credit: JPL </center>

Going back to WASP-12b, the planet's nightside temperature was found to be 1340 K [(Bell et al., 2019)](https://arxiv.org/pdf/1906.04742.pdf), giving a planet-to-star flux ratio of 0.0743%. The offset in the brightest point in the phasecurve from the centre of eclipse was also found to be 19.1 degrees east. To make our planet model have a nightside flux ratio of 0.0743%, a dayside flux ratio of 0.3917%, and a phase offset of 19.1 degrees east, we'll need to set ```planet[1,0]``` and ```planet[1,1]``` to some non-zero value. This will increase the dayside flux and decrease the nightside flux, so to keep the same eclipse depth we'll also need to renormalize ```planet.L```.

To get us ready for fitting real data soon, we're going to do this by fitting our $\texttt{starry}$ model to the simple sinusoidal model I used in my paper. I'll generate the data that you need to fit below:

In [ ]:
tstart = planet.tref-planet.porb/2-0.1*planet.porb # Start just before eclipse
tend = tstart + planet.porb # End one orbital period later
time = np.linspace(tstart, tend, 1000)

# Taking fitted values from my Bell+2019 paper
phasecurve = 0.003917*(1+0.395*(-1+np.cos(2*np.pi*(time-planet.tref-planet.porb/2.)/planet.porb))
                        -0.136*np.sin(2*np.pi*(time-planet.tref-planet.porb/2.)/planet.porb))

# Set the flux during eclipse to zero
eclDuration = 178./60./24.
inEclipse = np.logical_and(time<planet.tref-planet.porb/2+eclDuration/2, time>planet.tref-planet.porb/2-eclDuration/2)
phasecurve[inEclipse] = 0

plt.plot((time-planet.tref)*24*60, phasecurve)
plt.title('Provided model of WASP-12b\'s flux variations (ignoring transit)')
plt.xlabel("Time from transit (minutes)")
plt.ylabel("System Flux")
plt.show()

### Question #3)

Fit the ```planet``` parameters to the data provided in my array ```phasecurve```. To do this, use the function ```scipy.optimize.minimize```, and make an initial guess of $\theta_0$=[0.004, 0.4, 0.1] for ```planet.L```, ```planet[1,0]```, and ```planet[1,1]``` respectively. The function that you want ```scipy``` to minimize should take in an intial guess $\theta_0$, use that to update the three ```planet``` parameters, recompute the phasecurve by doing ```system.compute(time)```, and then return the mean squared error (MSE) between ```planet.lightcurve``` and my ```phasecurve``` array. You should also set ```method='Nelder-Mead'``` when calling ```scipy.optimize.minimize``` as it will work better for this data than the default method.

Get the best-fit parameters and update your ```planet``` parameters. Make a plot showing that your fitted ```planet.lightcurve``` is pretty close to my ```phasecurve``` array.

In [ ]:
# Your code goes here




# Detector Systematics and Decorrelation

Unfortunately, no phasecurve that we collect using Spitzer looks anywhere near that clean. Imperfections in our telescopes and/or detectors cause strong correlated noise in our data—we call this detector systematics.

For the IRAC channel 1 and channel 2 cameras on Spitzer, the dominant source of detector systematics is intra-pixel sensitivity variations. Each individual pixel is a square piece of semiconductor that is made up of indium and antimony. Because we are not able to make these pixels perfectly, we end up with pixels that are more sensitive to incoming photons in the middle of the pixel and less sensitive to photons near the edges of the pixel. There are also gaps between pixels where no light is collected at all.

As Spitzer is a space telescope, it is free-floating and is prone to oscillations in the direction it is pointing because it has nothing it can secure itself to. While this shaking motion is controlled enough for most uses of the telescope, this shaking drifts the starlight around by a signficant fraction of a pixel. This results in fairly rapid fluctuations in brightness that are of a similar strength as our astrophysical signal. For example, here is the raw data showing the raw flux from the system and the position of the star on the detector over time.

<img src='images/raw_WASP12b_lightcurve.png' width=100%>
<center>Image Credit: Bell et al. 2019</center>

While the transit is visible by eye, all other obvious signals are caused by detector systematics. However, all hope is not lost! You can clearly see that there is a very strong and repeatable correlation between the position of the star on the detector and the flux we measure. By trying to model this correlation, we can try to measure the underlying astrophysical signal that we are interested in. This is where things get tough though... Every research group has their own idea of how to best model these detector systematics and firmly believes everybody else does it wrong. The more likely truth, we're all modelling these systematics wrong to an extent and each technique probably has its strengths and weaknesses.

To dip our toes into this fun but challenging world, we are going to code up the simplest possible detector model and try fitting real, previously published Spitzer data!

### Question 4 Setup

First we need to load in the data that we will fit. I've given you the real Spitzer data for the exoplanet CoRoT-2b. I have also written some code that will load in the data for that phasecurve for you.

In [ ]:
flux, time, x, y = helper.get_data('CoRoT-2b_ch2_datacube_binned.dat')

fig, axs = plt.subplots(3, sharex=True, sharey=False, gridspec_kw={'hspace': 0})
axs[0].plot(time, flux, '.')
axs[1].plot(time, x, '.')
axs[2].plot(time, y, '.')
axs[0].set_ylabel('Raw Flux')
axs[1].set_ylabel('x')
axs[2].set_ylabel('y')
axs[2].set_xlabel('Time (BJD)')
plt.show()

### Question 4a)

Make a new planet variable here (with lmax=1). Initialize it with the values you find at http://exoplanet.eu/catalog/corot-2_b/

In [ ]:
# Your code goes here




### Question 4b)

Write up a simple function to model the sensitivity of the detector where the sensitivity is a second-order,2-dimensional polynomial as a function of the $x$ and $y$ position of the star. Second order means that the function includes terms up to $x^2$, $y^2$, and all the cross-terms (6 total coefficients). You can write this from scratch or find a scipy/numpy function or some other function that does it for you, but make sure to acknowledge the source of your code if you did not write it yourself.

In [ ]:
# Your code goes here




### Question 4c)

Now write a function that puts all of our work together. The parameters that you are going to be fitting for are the two stellar limb darkening parameters (```star[1]``` and ```star[2]```), the ```planet.r``` parameter (planet-to-star radius ratio), the ```planet.L``` parameter, the ```planet[1,0]``` and ```planet[1,1]``` parameters, and the 6 polynomial coefficients.

Your function should take 5 inputs: the $\theta_0$ guess holding your 12 fitted parameters, the time, the x and y arrays, the ```star``` variable, and the ```planet``` variable. It should use the $\theta_0$ parameter to update all of the relevant components of ```star``` and ```planet```, make a ```system```, run the function ```system.compute```, and then multiply ```system.lightcurve``` by the sensitivity model you wrote for the previous part.

In [ ]:
# Your code goes here




### Question 4d)

Now write a log-probability function that evaluates the fit of the model. The function should take all of the same inputs as the function you wrote for Question 4c, but $\theta_0$ will contain one extra parameter—the $\sigma$ parameter describing the white noise in the data; we have to fit for this since we don't know what it should be given our data.

Feel free to look back at your Lab 08 work or the Search and Rescue files if you need to remember how to write one of these functions.

In addition to the usual log-probability term you used last week when fitting a line, we will also need to penalize the model for choosing an overly large value for $\sigma$ (this is similar to seeking a reduced chi-squared of 1). Your log-probability function should have the form

\begin{equation}
    -0.5\chi^2-N_{\rm data}\log(\sigma)
\end{equation}

As for a prior, you should:
1. constrain the planet's radius to be positive and smaller 1
2. constrain the planet's luminosity to be positive and smaller than 1
3. constrain the planet's surface brightness to be positive everywhere (negative flux makes no sense). This can be checked using ```planet.is_physical()``` <u>after</u> having updated the planet's parameters.

In [ ]:
# Your code goes here




### Question 4e)

Make an initial guess $\theta_0$, setting rp to ~0.1, your L to ~1e-3, your ```planet[1,0]``` parameter to ~0.4, your ```planet[1,1]``` parameter to ~0, your q1 and q2 terms to ~0.1, your constant polynomial term to 1.0, your other polynomial terms to ~0.01, and your white noise parameter to ~1e-3. These guesses will get us in the right ballpark, but aren't great guesses.

Plot your initial guess against the data to make sure it looks vaguely reasonable.

Make a dense Gaussian ball around these initial guesses so that each MCMC walker will start in its own position in every dimension. Something like a spread of 1e-5 times the initial guess should work, but also add a random number on the order of ~1e-5 to spread out any values that were started around zero (where multiplying by a random number does nothing). Then run a burn-in MCMC chain using ```emcee``` like you learned in the last lab. The syntax will be 

```state = sampler.run_mcmc(pos0, nsteps, progress=True)```

I would use 50 walkers and take 2000 steps (for a total number of 10000 log-likelihood evaluations). After this many steps, the MCMC should have converged. Discard these steps using ```sampler.reset()```, and take an additional 2000 steps starting from the same point that you ended off on using the syntax

```state = sampler.run_mcmc(state, nsteps, progress=True)```

where state is the output of the first ```run_mcmc``` function call.

In [ ]:
# Your code goes here




### Question 4f)

Now it's time to examine your results.

1. Plot ```sampler.lnprobability.T``` to see how your log-likelihood evolved over the MCMC production run. It should look like a bunch of squiggly lines with no steep slope on the left side of the plot or any obvious trends. If there are any obvious trends, your MCMC hasn't finished burning in; re-run your two MCMCs using a larger number of steps for the first one to ensure you have burned in the model.

2. Print your fitted parameters from the chain (use the median value) and their associated uncertainties (you can assume that they are Gaussian). Make sure to also print the name of each parameter beside the value and uncertainty so it is clear what values belong to what parameters.

3. Plot your fitted model over top of your raw data, and plot your raw data with the errorbars you found during your fitting.

4. Plot your fitted model divided by your fitted sensitivity model (aka just the astrophysical signal) over top of your raw data divided by your fitted sensitivity model. Now you should clearly be able to see the transit, eclipses, and phase variations.

In [ ]:
# Your code goes here




### Question 4g)

Compute the offset in degrees of the peak in the phasecurve from the phase of eclipse using 

```offset = np.arctan2(planet[1,1], planet[1,0])*180/np.pi```.

Positive offsets mean the phasecurve peaks before eclipse. Using this knowledge, what direction are the winds on your planet blowing (east or west).

In [ ]:
# Your code goes here




### Question 4h)

Compute the dayside and nightside temperatures of the planet by inverting the Planck function. I have provided a function to do this below. All you need to do is pass in the 3 required parameters for each hemisphere.

In [ ]:
def inversePlanck(fp_fstar, rp_rs, Tstar):
    """Invert the Planck function to find the apparent temperature of observed hemisphere.
    
    Args:
        fp_fstar (float): The planet-to-star flux ratio, found using planet.flux(𝜓)*planet.L,
            where 𝜓=0 for the dayside and 𝜓=180 for the nightside
        rp_rs (float): The fitted value of planet.r
        Tstar (float): The stellar effective temperature found at exoplanet.eu
        
    Returns:
        float: The apparent temperature of observed hemisphere in Kelvin.
    
    """
    
    wav = 4.5e-6 # m
    return const.h.value*const.c.value/(const.k_B.value*wav)*(np.log(1+(np.exp(const.h.value*const.c.value/(const.k_B.value*wav*Tstar))-1)/(fp_fstar/rp_rs**2)))**-1

In [ ]:
# Your code goes here


